In [1]:
# libraries
import psycopg2 as pg
import pandas as pd
import numpy as np
import os

# set environment variable for psycog2 (for some systems)
os.environ["PGGSSENCMODE"] = "disable"

# get the stored passwords
f=open("credentials_david.txt", "rt")
pwd=f.readline().strip()  #di   password
hpwd=f.readline().strip() #home password
f.close()

In [2]:
# di test: WORKING
#host_di = "appserver-01.alunos.di.fc.ul.pt"
#db_di = "tpd012"
#user_di = "tpd012"

#conn = pg.connect(host = host_di,
#                  database = db_di,
#                  user = user_di,
#                  password = pwd)
#conn.close()

# local test: WORKING
host_local = "localhost"
db_local = "tpd012"
user_local = "postgres"

conn = pg.connect(host = host_local,
                  database = db_local,
                  user = user_local,
                  password=hpwd)
conn.close()

In [3]:
# functions
def run_sql_command(sql, host, database, user, password):
    """Executes a single SQL statement from a string variable and the database credentials"""
    conn = pg.connect(host = host,
                      database = database,
                      user = user,
                      password = password)
    cur = conn.cursor()
    cur.execute(sql)
    cur.close()
    conn.commit()
    conn.close() 
    
def intTryParse(value):
    """Parse a string to an integer"""
    try:
        a = int(value)
        return True
    except ValueError:
        return False

In [4]:
listings_al_file_path = '../data/listings_al.csv'
df_listings_al = pd.read_csv(listings_al_file_path)

C:\Users\david\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 2.2. Preprocessing

The defined _'Property'_ dimension table in __Phase I__ is as follows:

<img src="property_schema.png" width="150" align="center"/>

# __CHANGE SCHEMA__

In [5]:
def delete_null_rows(df,columns):
    """Removes all records with any empty cells from input DataFrame"""
    processed_df = df[columns].copy()
    total_rows = processed_df.shape[0]
    delete_rows = []
    
    if processed_df.isnull().values.any(): # if there are any null values in DataFrame, process DataFrame
        for index, row in enumerate(processed_df.itertuples(), start = 0):
            if (pd.Series(row).isnull().values.any()): # if row has any null value
                delete_rows.append(index) # add row index to delete list
    
    processed_df.drop(df.index[delete_rows], inplace = True) # delete rows fr
    processed_df = processed_df.reset_index().drop('index', axis = 1)
    print('DataFrame contains {} rows. Deleted {} rows ({}% of total rows)'.format(processed_df.shape[0], len(delete_rows), round(len(delete_rows)*100/total_rows, 2)))
    return processed_df

In [6]:
df_columns = ['id',
              'property_type',
              'room_type',
              'accommodates',
              'bathrooms',
              'bedrooms',
              'beds',
              'bed_type']

df_property = delete_null_rows(df_listings_al,df_columns)

DataFrame contains 17097 rows. Deleted 71 rows (0.41% of total rows)


`df_property` will be the table containing a record for each fact ID converted to the dimension format. `df_property_dimension` will be the _de facto_ property dimension from which we can convert each fact ID record to the dimension key according to its set of attributes.

### _'property_type_category'_

We start by creating _'property_type_category'_, a set of four categories from _'property_type'_.

In [7]:
category = []
# change 'property_type' to four categories, exclude exceptions
for i, value in enumerate(df_property['property_type']):
    if value in ["Apartment", "Serviced apartment", "Aparthotel","Loft"]: # 4
        category.append('Apartment')
    elif value in ["House","Townhouse","Villa","Dome house","Vacation home","Lighthouse","Casa particular (Cuba)","Tiny house","Farm stay","Cottage"]: # 10
        category.append('House')
    elif value in ["Guesthouse","Guest suite"]: # 2
        category.append('Guesthouse')
    elif value in ["Hostel","Bed and breakfast","Boutique hotel","Hotel"]: # 4
        category.append('Hotel/Hostel')
    else:
        category.append(None) # for the case of missing or invalid values
        
df_property['property_type_category'] = category

### _'room_type'_

This attribute is already defined in satisfactory categories.

### _'accommodates'_

In [8]:
# change 'property_type' to four categories, exclude exceptions
for i, value in enumerate(df_property['accommodates']):
    if value in [1,2]:  
        df_property.loc[i, 'accommodates'] = 'Up to 2'
    elif value in [3,4]:
        df_property.loc[i, 'accommodates'] = 'Up to 4'
    elif value in [5,6]:
        df_property.loc[i, 'accommodates'] = 'Up to 6'
    else:
        df_property.loc[i, 'accommodates'] = 'Up to 7 or more'
        # no missing values in this column

### _'bathrooms'_

In [9]:
# change 'bathrooms' to five categories, exclude exceptions
for i, value in enumerate(df_property['bathrooms']):
    if value < 0.5:  
        df_property.loc[i, 'bathrooms'] = 'No bathrooms'
    elif value < 1.5:
        df_property.loc[i, 'bathrooms'] = '1 bathroom'
    elif value < 2.5:
        df_property.loc[i, 'bathrooms'] = '2 bathrooms'
    elif value < 3.5:
        df_property.loc[i, 'bathrooms'] = '3 bathrooms'
    elif value >= 3.5:
        df_property.loc[i, 'bathrooms'] = '4+ bathrooms'
    else:
        df_property.loc[i, 'bathrooms'] = None # for the case of missing or invalid values

### _'bedrooms'_

In [10]:
# change 'bedrooms' to five categories, exclude exceptions
for i, value in enumerate(df_property['bedrooms']):
    if value == 0:  
        df_property.loc[i, 'bedrooms'] = 'T0'
    elif value == 1:
        df_property.loc[i, 'bedrooms'] = 'T1'
    elif value == 2:
        df_property.loc[i, 'bedrooms'] = 'T2'
    elif value == 3:
        df_property.loc[i, 'bedrooms'] = 'T3'
    elif value >= 4:
        df_property.loc[i, 'bedrooms'] = 'T4+'
    else:
        df_property.loc[i, 'bedrooms'] = None # for the case of missing or invalid values

### _'beds'_

In [11]:
# change 'beds' to five categories, exclude exceptions
for i, value in enumerate(df_property['beds']):
    if value == 0:  
        df_property.loc[i, 'bedrooms'] = 'No beds'
    elif value == 1:
        df_property.loc[i, 'bedrooms'] = '1 bed'
    elif value == 2:
        df_property.loc[i, 'bedrooms'] = '2 beds'
    elif value == 3:
        df_property.loc[i, 'bedrooms'] = '3 beds'
    elif value >= 4:
        df_property.loc[i, 'bedrooms'] = '4+ beds'
    else:
        df_property.loc[i, 'bedrooms'] = None # for the case of missing or invalid values

### Deleting missing data

Besides the initial removal of rows that had one or more missing data cells, we now remove any cells that may now have missing data from our recategorisation process.

In [12]:
df_property = delete_null_rows(df_property,df_property.columns)

df_property = df_property[['id',
              'property_type_category',
              'property_type',
              'room_type',
              'accommodates',
              'bathrooms',
              'bedrooms',
              'beds',
              'bed_type']]
df_property

DataFrame contains 16626 rows. Deleted 471 rows (2.75% of total rows)


,id,property_type_category,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type
0,25659,Apartment,Apartment,Entire home/apt,Up to 4,1 bathroom,1 bed,1.0,Real Bed
1,29248,Apartment,Apartment,Entire home/apt,Up to 4,1 bathroom,2 beds,2.0,Real Bed
2,29396,Apartment,Apartment,Entire home/apt,Up to 4,1 bathroom,1 bed,1.0,Real Bed
3,29720,Apartment,Apartment,Entire home/apt,Up to 7 or more,4+ bathrooms,4+ beds,14.0,Real Bed
4,27899698,Apartment,Apartment,Entire home/apt,Up to 4,2 bathrooms,2 beds,2.0,Real Bed
...,...,...,...,...,...,...,...,...,...
16621,41870065,Apartment,Apartment,Entire home/apt,Up to 2,1 bathroom,1 bed,1.0,Real Bed
16622,41879410,Apartment,Apartment,Entire home/apt,Up to 4,1 bathroom,3 beds,3.0,Real Bed
16623,41882911,Apartment,Apartment,Entire home/apt,Up to 4,1 bathroom,3 beds,3.0,Real Bed
16624,41879552,Apartment,Loft,Entire home/apt,Up to 2,1 bathroom,1 bed,1.0,Real Bed


In this step, a larger number of rows was lost, but still a small percentage in relation to the entirety of the dataset.

## 2.3. Creating the dimension table

In [13]:
# create df_property_dimension
df_property_dimension = df_property[['property_type_category','property_type','room_type','accommodates','bathrooms','bedrooms','beds','bed_type']].drop_duplicates().copy()
df_property_dimension = df_property_dimension.reset_index().drop('index', axis = 1)

df_property_dimension

,property_type_category,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type
0,Apartment,Apartment,Entire home/apt,Up to 4,1 bathroom,1 bed,1.0,Real Bed
1,Apartment,Apartment,Entire home/apt,Up to 4,1 bathroom,2 beds,2.0,Real Bed
2,Apartment,Apartment,Entire home/apt,Up to 7 or more,4+ bathrooms,4+ beds,14.0,Real Bed
3,Apartment,Apartment,Entire home/apt,Up to 4,2 bathrooms,2 beds,2.0,Real Bed
4,Apartment,Apartment,Entire home/apt,Up to 4,1 bathroom,No beds,0.0,Real Bed
...,...,...,...,...,...,...,...,...
967,Apartment,Apartment,Private room,Up to 7 or more,4+ bathrooms,4+ beds,7.0,Real Bed
968,Hotel/Hostel,Bed and breakfast,Private room,Up to 6,2 bathrooms,3 beds,3.0,Real Bed
969,Apartment,Serviced apartment,Entire home/apt,Up to 4,3 bathrooms,2 beds,2.0,Real Bed
970,Guesthouse,Guesthouse,Shared room,Up to 7 or more,3 bathrooms,4+ beds,8.0,Real Bed


## 2.4. Attributing dimension keys to facts

Having our dimension DataFrame ready, we can now use our converted listings table, `df_property` (which contains Property dimension attributes for all valid facts), to assign each fact its corresponding Property dimension foreign key.

In [14]:
# change df_property to property dimension ID
FK = []
for i, row in enumerate(df_property.itertuples(), start = 0): # creates enumerate object (counter, value1, value2, ...)
    row = list(row[2:]) # remove ID and index
    for j, dim_row in enumerate(df_property_dimension.itertuples(), start = 0):
        dim_row = list(dim_row[1:]) # remove index
        if row == dim_row: # checks which dimension row the fact is equal to
            FK.append(j)

In [15]:
df_property['Property'] = FK

In [16]:
df_property = df_property[['id','Property']]
df_property

,id,Property
0,25659,0
1,29248,1
2,29396,0
3,29720,2
4,27899698,3
...,...,...
16621,41870065,5
16622,41879410,8
16623,41882911,8
16624,41879552,41


## 2.5. Creating the _Property_ dimension table in SQL

The SQL statement to create the table is as follows:

In [17]:
create_table = """
CREATE TABLE Property (
    PROPERTY_ID SERIAL PRIMARY KEY NOT NULL,
    PROPERTY_TYPE_CATEGORY VARCHAR(30) CHECK (PROPERTY_TYPE in ('Apartment', 'Guesthouse', 'House', 'Hotel/Hostel')) NOT NULL,
    PROPERTY_TYPE VARCHAR(30) CHECK (PROPERTY_TYPE in ('Apartment','Serviced apartment','Aparthotel','Loft','House','Townhouse','Villa','Dome house','Vacation home','Lighthouse','Casa particular (Cuba)','Tiny house','Farm stay','Cottage','Guesthouse','Guest suite','Hostel','Bed and breakfast','Boutique hotel','Hotel')) NOT NULL,
    ROOM_TYPE VARCHAR(30) CHECK (ROOM_TYPE in ('Entire Property', 'Private Room', 'Hotel Room', 'Shared Room')) NOT NULL,
    ACCOMMODATES VARCHAR(10) CHECK (ACCOMMODATES in ('Up to 2','Up to 4','Up to 6','Up to 7 or more')) NOT NULL,
    BATHROOMS VARCHAR(10) CHECK (BATHROOMS in ('No bathrooms','1 bathroom','2 bathrooms','3 bathrooms','4+ bathrooms')) NOT NULL,
    BEDROOMS VARCHAR(10) CHECK (BEDROOMS in ('T0','T1','T2','T3','T>=4')) NOT NULL,
    BEDS VARCHAR(10) CHECK (BATHROOMS in ('No beds','1 bed','2 beds','3 beds','4+ beds')) NOT NULL,
    BED_TYPE VARCHAR(30) CHECK (BED_TYPE in ('Real Bed', 'Pull-out Sofa', 'Futton', 'Couch', 'Airbed')) NOT NULL,
)
"""

In [18]:
# creating table
# run_sql_command(create_table,
#                host_local,
#                db_local,
#                user_local,
#                hpwd)

The `create_table` command contains the integrity constraints essential for modelling the dimension.

The dimension `Property` is ready for data __loading__. Now, it's necessary to __transform__ the data, which was previously __extracted__ from its source.